In [1]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading
import pickle 
from pydrake.all import HPolyhedron
world = Village()
village_side = 39
village_height = 10
world.build(village_height=village_height, village_side=village_side, building_every=5, density=0.15)

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:

path = 'logs/experiment_village_redu_39_10_08_11_17_51_59___0_500_0.050_0.200/data/it_17.pkl'
with open(path, 'rb') as f:
   d =  pickle.load(f)

In [3]:
ra = []
rb = []
for ga,gb in zip(d['ra'], d['rb']):
    ra+=ga
    rb+=gb
regions = [HPolyhedron(a,b) for a,b in zip(ra, rb)]

In [4]:
world.plot_regions(regions)

In [5]:
from dijkstraspp import DijkstraSPPsolver

In [6]:
def conv(pt):
    return pt

dspp = DijkstraSPPsolver(regions, conv)

[DijkstraSPP] Pre-Building adjacency matrix  0 / 115
[DijkstraSPP] Pre-Building adjacency matrix  10 / 115
[DijkstraSPP] Pre-Building adjacency matrix  20 / 115
[DijkstraSPP] Pre-Building adjacency matrix  30 / 115
[DijkstraSPP] Pre-Building adjacency matrix  40 / 115
[DijkstraSPP] Pre-Building adjacency matrix  50 / 115
[DijkstraSPP] Pre-Building adjacency matrix  60 / 115
[DijkstraSPP] Pre-Building adjacency matrix  70 / 115
[DijkstraSPP] Pre-Building adjacency matrix  80 / 115
[DijkstraSPP] Pre-Building adjacency matrix  90 / 115
[DijkstraSPP] Pre-Building adjacency matrix  100 / 115
[DijkstraSPP] Pre-Building adjacency matrix  110 / 115
[DijkstraSPP] Pre-Building d-adjacency matrix  0 / 491


In [7]:
start= np.zeros((3,1))
end = np.array([village_side,village_side, 4]) 
print(point_in_regions(start,regions ))
print(point_in_regions(end,regions ))

True
True


In [8]:
def densify_waypoints(waypoints):
    densify = 20
    dists = []
    dense_waypoints = []
    for idx in range(len(waypoints[:-1])):
        a = waypoints[idx]
        b = waypoints[idx+1]
        t = np.linspace(1,0, 10)
        locs = []
        dists = []
        for tval in t:
            pa = a*tval + b*(1-tval)
            locs.append(pa)
        for i in range(len(locs)-1):
            dists.append(np.linalg.norm(locs[i]- locs[i+1]))
        d = np.sum(dists)
        #print(d * densify)
        t = np.linspace(1,0,int(densify*d))
        for tval in t:
            dense_waypoints.append(waypoints[idx]*tval + waypoints[idx+1]*(1-tval))
    return dense_waypoints



In [9]:
wp_sparse, _ = dspp.solve(start.squeeze(), end.squeeze())
wp_dense = densify_waypoints(wp_sparse)
world.plot_points(wp_dense)

[DijkstraSPP] optimized distance/ start-distance = 63.14 / 70.81 = 0.89
